In [ ]:
import sys
sys.path.append('../')
sys.path.append('../../')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

def generate_data(mean, cov,
    n=1000,
    ):
    data = np.random.multivariate_normal(mean, cov, n) # (n,2)
    x = data[:, 0]
    y = data[:, 1]
    return x, y

def kde_functional(x, y, xlim, ylim,
    kde_n=500,
    kde_factor=.25,
    ):
    xmin, xmax = xlim
    ymin, ymax = ylim
    xx, yy = np.meshgrid(np.linspace(xmin, xmax, kde_n), np.linspace(ymin, ymax, kde_n))
    positions = np.vstack([xx.ravel(), yy.ravel()])
    values = np.vstack([x, y])
    kernel = st.gaussian_kde(values, kde_factor)
    f = np.reshape(kernel(positions).T, xx.shape) # (m,m)
    f = f/np.sum(f)
    return xx, yy, f

def get_local_kl(p, q,
    eps=0,
    ):
    local_kl = p*np.log(p/(q+eps))
    return local_kl

### generate data
xlim = [-5, 5]
ylim = [-5, 5]

m = .5
x1, y1 = generate_data([-m, -m], [[0.8, 0.05], [0.05, 0.7]]) # (n),(n)
x2, y2 = generate_data([m, m], [[0.8, 0.05], [0.05, 0.5]]) # (n),(n)
_x2, _y2 = generate_data([-2, 1], [[.01, 0], [0, .01]], n=50) # (n),(n)
x2 = np.concatenate([x2, _x2], axis=0)
y2 = np.concatenate([y2, _y2], axis=0)

### kernel density estimate
xx1, yy1, f1 = kde_functional(x1, y1, xlim, ylim) # (m,m)
xx2, yy2, f2 = kde_functional(x2, y2, xlim, ylim) # (m,m)

### kullback-leibler and jensen-shanon
p = f1
q = f2
local_kl_pq = get_local_kl(p, q)
local_kl_qp = get_local_kl(q, p)
local_js =1/2*get_local_kl(p, (p+q)/2)+1/2*get_local_kl(q, (p+q)/2)
print(np.min(local_js), np.max(local_js))

### plots
s = 15

for f_div in [local_kl_pq, local_kl_qp, local_js]:
    fig, ax = plt.subplots(1, 1, figsize=(20,10))
    #cset = ax.contour(xx1, yy1, local_js, colors='k')
    #ax.clabel(cset, inline=.5, fontsize=10)
    cfset = ax.contourf(xx1, yy1, f_div, cmap='inferno_r' , alpha=1)

    cset = ax.contour(xx1, yy1, f1, colors='r')
    ax.clabel(cset, inline=1, fontsize=10)
    ax.scatter(x1, y1, c='r', s=s, lw=0, alpha=.5)

    cset = ax.contour(xx2, yy2, f2, colors='b')
    ax.clabel(cset, inline=1, fontsize=10)
    ax.scatter(x2, y2, c='b', s=s, lw=0, alpha=.5)

    ax.set_xlim(*xlim)
    ax.set_ylim(*ylim)
    plt.show()